In [7]:
from dynamiq import Workflow
from dynamiq.connections import ScaleSerp
from dynamiq.flows import Flow
from dynamiq.nodes.agents.base import Agent
from dynamiq.nodes.agents.orchestrators.graph import END, START, GraphOrchestrator
from dynamiq.nodes.agents.orchestrators.graph_manager import GraphAgentManager
from dynamiq.nodes.agents.react import ReActAgent
from dynamiq.nodes.tools.scale_serp import ScaleSerpTool
from dynamiq.utils.logger import logger
from llm_setup import setup_llm
import textwrap

# Please use your own file path
OUTPUT_FILE_PATH = "trip.md"

In [8]:

def _validate_input_data(input_data):
    """
    Validate the input data dictionary for required keys.

    Args:
        input_data (dict): The input data dictionary to validate

    Raises:
        KeyError: If any required key is missing from the input_data dictionary
    """
    required_keys = ["dates", "location", "cities", "interests"]
    for key in required_keys:
        if key not in input_data:
            raise KeyError(f"Missing required key: {key}")


def _format_prompt(template, input_data):
    """
    Format the prompt template with input data.

    Args:
        template (str): The prompt template string
        input_data (dict): The input data dictionary

    Returns:
        str: The formatted prompt string
    """
    return textwrap.dedent(template).format(**input_data)


def generate_customer_prompt(input_data):
    """
    Generate a detailed customer prompt for comprehensive trip planning.

    This function creates a prompt that instructs to analyze and select the best city,
    compile an in-depth city guide, and create a 7-day travel itinerary.

    Args:
        input_data (dict): A dictionary containing trip information with the following keys:
            - dates (str): The dates of the trip
            - location (str): The traveler's starting location
            - cities (str): A list of potential city options
            - interests (str): The traveler's interests

    Returns:
        str: A formatted string containing the detailed customer prompt

    Raises:
        KeyError: If any required key is missing from the input_data dictionary
    """
    _validate_input_data(input_data)

    template = """
    Analyze and select the best city for the trip based on weather patterns, seasonal events, and travel costs.
    Compare multiple cities considering current weather, upcoming events, and travel expenses. Provide a detailed
    report on the chosen city, including flight costs, weather forecast, and attractions.

    Next, compile an in-depth city guide with key attractions, local customs, events, and daily activity recommendations.
    Include hidden gems, cultural hotspots, must-visit landmarks, weather forecasts, and costs. The guide should be rich
    in cultural insights and practical tips to enhance the travel experience.

    Finally, expand the guide into a 7-day travel itinerary with detailed daily plans, including weather forecasts,
    places to eat, packing suggestions, and a budget breakdown. Suggest actual places to visit, hotels, and restaurants.
    The itinerary should cover all aspects of the trip, from arrival to departure, with a daily schedule, recommended
    clothing, items to pack, and a detailed budget.

    Trip Date: {dates}
    Traveling from: {location}
    City Options: {cities}
    Traveler Interests: {interests}
    """  # noqa: E501

    return _format_prompt(template, input_data)

In [9]:
def inference(input_data: dict) -> dict:
    llm_agent = setup_llm()
    http_connection_serp = ScaleSerp()
    tool_search = ScaleSerpTool(connection=http_connection_serp)

    # Create agents
    agent_selection_city = ReActAgent(
        name="City Selection Expert",
        role="An expert in analyzing travel data to pick ideal destinations",
        goal=(
            "help select the best city for a trip based on specific criteria such as weather patterns, seasonal events, and travel costs."  # noqa: E501
        ),
        llm=llm_agent,
        tools=[tool_search],
        max_loops=15,
    )

    agent_city_guide = ReActAgent(
        name="City Guide Expert",
        role="An expert in gathering information about a city",
        goal=(
            "compile an in-depth guide for someone traveling to a city, including key attractions, local customs, special events, and daily activity recommendations."  # noqa: E501
        ),
        llm=llm_agent,
        tools=[tool_search],
        max_loops=15,
    )

    agent_writer = Agent(
        name="City Guide Writer",
        role="An expert in creating detailed travel guides",
        goal="write a detailed travel guide for a city, including key attractions, local customs, special events, and daily activity recommendations.",  # noqa: E501
        llm=llm_agent,
    )

    agent_manager = GraphAgentManager(
        llm=llm_agent,
    )

    # Create a linear orchestrator
    orchestrator = GraphOrchestrator(
        manager=agent_manager,
        final_summarizer=True,
    )

    orchestrator.add_node("select_city", [agent_selection_city])
    orchestrator.add_node("gather_city_information", [agent_city_guide])
    orchestrator.add_node("document_infromation", [agent_writer])

    orchestrator.add_edge(START, "select_city")
    orchestrator.add_edge("select_city", "gather_city_information")
    orchestrator.add_edge("gather_city_information", "document_infromation")
    orchestrator.add_edge("document_infromation", END)

    # Create a workflow
    workflow = Workflow(flow=Flow(nodes=[orchestrator]))

    user_prompt = generate_customer_prompt(input_data)

    print(user_prompt)
    result = workflow.run(
        input_data={
            "input": user_prompt,
        }
    )
    logger.info("Workflow completed")
    content = result.output[orchestrator.id]
    return content


if __name__ == "__main__":
    user_location = input("Enter your location: ")
    user_cities = input("Enter cities you want to visit: ")
    user_dates = input("Enter dates: ")
    user_interests = input("Enter your interests: ")
    input_data = {
        "location": user_location,
        "cities": user_cities,
        "dates": user_dates,
        "interests": user_interests,
    }
    content = inference(input_data)["output"]["content"]
    print(content)
    with open(OUTPUT_FILE_PATH, "w") as f:
        f.write(content)

2024-11-09 20:49:11 - INFO - Workflow 88275df3-39b8-4ae8-8ae4-39115a077f83: execution started.
2024-11-09 20:49:11 - INFO - Flow efb95098-653d-4d4d-bea1-5f623c9083e4: execution started.
2024-11-09 20:49:11 - INFO - Node GraphOrchestrator - a02625de-f246-4b2a-98b2-e4cbf51f3de3: execution started.
2024-11-09 20:49:11 - INFO - Node Graph Manager - 34a014f5-cda2-45e2-bc59-18a3c722882d: execution started.
2024-11-09 20:49:11 - INFO - AgentManager Graph Manager - 34a014f5-cda2-45e2-bc59-18a3c722882d: started with input {'action': 'assign', 'task': ('Name:', 'City Selection Expert', 'Role:', 'An expert in analyzing travel data to pick ideal destinations'), 'chat_history': [{'role': 'user', 'content': '\nAnalyze and select the best city for the trip based on weather patterns, seasonal events, and travel costs.\nCompare multiple cities considering current weather, upcoming events, and travel expenses. Provide a detailed\nreport on the chosen city, including flight costs, weather forecast, and a


Analyze and select the best city for the trip based on weather patterns, seasonal events, and travel costs.
Compare multiple cities considering current weather, upcoming events, and travel expenses. Provide a detailed
report on the chosen city, including flight costs, weather forecast, and attractions.

Next, compile an in-depth city guide with key attractions, local customs, events, and daily activity recommendations.
Include hidden gems, cultural hotspots, must-visit landmarks, weather forecasts, and costs. The guide should be rich
in cultural insights and practical tips to enhance the travel experience.

Finally, expand the guide into a 7-day travel itinerary with detailed daily plans, including weather forecasts,
places to eat, packing suggestions, and a budget breakdown. Suggest actual places to visit, hotels, and restaurants.
The itinerary should cover all aspects of the trip, from arrival to departure, with a daily schedule, recommended
clothing, items to pack, and a detailed b

2024-11-09 20:49:14 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-09 20:49:14 - INFO - Node OpenAI LLM - f79c8c53-7de3-4e20-a916-9e9c608e06b8: execution succeeded in 3.2s.
2024-11-09 20:49:14 - INFO - Node Graph Manager - 34a014f5-cda2-45e2-bc59-18a3c722882d: execution succeeded in 3.2s.
2024-11-09 20:49:14 - INFO - Node City Selection Expert - 4ed8519d-7380-4b8a-a22f-bf954c22184b: execution started.
2024-11-09 20:49:14 - INFO - Agent City Selection Expert - 4ed8519d-7380-4b8a-a22f-bf954c22184b: Running ReAct strategy
2024-11-09 20:49:14 - INFO - Agent City Selection Expert - 4ed8519d-7380-4b8a-a22f-bf954c22184b: Loop 1 started.
2024-11-09 20:49:14 - INFO - Node OpenAI LLM - f79c8c53-7de3-4e20-a916-9e9c608e06b8: execution started.
2024-11-09 20:49:17 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-09 20:49:17 - INFO - Node OpenAI LLM - f79c8c53-7de3-4e20-a916-9e9c608e06b8: execution succ

# Final Report: Trip to Paris, France (December 15-17, 2024)

## Introduction
This report provides a comprehensive analysis and guide for your upcoming trip to Paris, France, from December 15 to December 17, 2024. The selection of Paris is based on favorable weather patterns, seasonal events, and reasonable travel costs, particularly for a traveler interested in history.

## 1. City Selection: Paris

### Weather Forecast
During your travel dates, Paris is expected to experience cold temperatures ranging from 4°C to 8°C (39°F to 46°F) with a likelihood of rain. It is advisable to pack warm clothing, including layers, a waterproof jacket, and comfortable walking shoes.

### Seasonal Events
In December, Paris transforms into a winter wonderland with numerous festive events:
- **Christmas Markets**: Various locations, including Saint-Germain-des-Prés and Notre Dame Cathedral, will host charming markets featuring local crafts, food, and holiday decorations.
- **Holiday Lights**: The city wi